## $\color{#dda}{\text{1: Libs (pmdarima)}}$

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import os
import pandas as pd
import pmdarima as pm
from pmdarima.utils import pacf, acf
import plotly.graph_objects as go
import warnings
warnings.filterwarnings('ignore')

## $\color{#dda}{\text{2: Creación de la serie de tiempo}}$

In [ ]:
df = pd.read_csv('observaciones.csv')

df['TIME_PERIOD'] = pd.to_datetime(df['TIME_PERIOD'], dayfirst = True)

fig = px.line(df, x='TIME_PERIOD', y='OBS_VALUE', title='Índice nacional de precios al consumidor quincenal')

fig.show()

## $\color{#dda}{\text{3: Visualizamos a ver qué hay}}$

## $\color{#dda}{\text{4: Bins}}$

## $\color{#dda}{\text{5: SARIMA}}$

## $\color{#dda}{\text{6: Error}}$